In [1]:
pip install pillow

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [112]:
from PIL import Image
import numpy as np

image_path = ('collage 10240.png')
image = Image.open(image_path)
test_array = np.random.rand(1000,1000)

image_np = np.array(image)

print(image_np.shape)

(7680, 10240, 3)


In [113]:
kernel_relief = np.array([[-2, -1, 0],
                          [-1,  1, 1],
                          [ 0,  1, 2]])



# Padding

In [114]:
import numpy as np

test = np.random.randint(1, 10, size = (4, 10))
arr2 = np.zeros((test.shape[0]+2, test.shape[1]+2), dtype=int)

for i in range(test.shape[0]):
    for j in range(test.shape[1]):
        arr2[i+1][j+1] = test[i,j]

print(test)
print('\n')
print(arr2)
print('\n')
print(f'test shape' + str(test.shape))
print(f'arr shape' + str(arr2.shape))

[[5 8 8 3 9 1 1 5 3 7]
 [6 2 2 2 2 8 4 5 2 1]
 [1 8 1 2 6 5 6 5 3 2]
 [8 1 5 5 9 5 1 5 7 7]]


[[0 0 0 0 0 0 0 0 0 0 0 0]
 [0 5 8 8 3 9 1 1 5 3 7 0]
 [0 6 2 2 2 2 8 4 5 2 1 0]
 [0 1 8 1 2 6 5 6 5 3 2 0]
 [0 8 1 5 5 9 5 1 5 7 7 0]
 [0 0 0 0 0 0 0 0 0 0 0 0]]


test shape(4, 10)
arr shape(6, 12)


# Свёртка

In [115]:
kernel_relief = np.array([[-2, -1, 0],
                          [-1,  1, 1],
                          [ 0,  1, 2]])
kernel_size = kernel_relief.shape[1]
test = np.random.randint(1, 10, size = (5, 10))

rows_num = test.shape[0]
column_num = test.shape[1]

padding = 0
stride = 1
output_height=((rows_num-kernel_size+2*padding)/stride)+1
output_width=((column_num-kernel_size+2*padding)/stride)+1

arr = np.zeros((int(output_height),int(output_width)), dtype=int)


print(test)
print('\n')
print(arr)
print('\n')
test.shape

[[4 1 6 2 4 3 2 9 9 2]
 [7 5 5 6 7 4 6 7 8 6]
 [9 3 1 6 8 9 2 1 9 8]
 [2 2 5 6 3 9 4 4 5 3]
 [4 7 7 5 2 8 8 7 2 3]]


[[0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0]]




(5, 10)

# Свёртка

In [103]:
test3 = np.random.randint(1,10, size = (3,3))
a = 0
for i in range(kernel_relief.shape[1]):
    for j in range(kernel_relief.shape[1]):
        a += test3[i][j]*kernel_relief[i][j]

print(test3)
print('\n')
print(kernel_relief)
print('\n')
a

[[3 1 6]
 [9 5 8]
 [4 3 1]]


[[-2 -1  0]
 [-1  1  1]
 [ 0  1  2]]




2

In [120]:

frame = np.zeros_like(kernel_relief)

i = j = k = l = val = 0
print(test)
print('\n')
print(arr)
print('\n')

while k < arr.shape[0]:
    while l < arr.shape[1]:
        while i < kernel_size:
            while j < kernel_size:
                frame[i][j] = test[i+k][j+l] # здесь заполнеяем один кадр изображения
                j += 1

            j = 0
            i += 1
        
        if (i == kernel_size): # как только кадр заполнился
            for m in range(frame.shape[0]):
                for n in range(frame.shape[1]):
                    val += frame[m][n]*kernel_relief[m][n]  # считаем значение свёртки
        arr[k][l] = val
        val = 0
        j = 0
        i = 0
        l += 1         

    l = 0
    k += 1


print(arr)
print('\n')

[[4 1 6 2 4 3 2 9 9 2]
 [7 5 5 6 7 4 6 7 8 6]
 [9 3 1 6 8 9 2 1 9 8]
 [2 2 5 6 3 9 4 4 5 3]
 [4 7 7 5 2 8 8 7 2 3]]


[[ -1  11  16  23   5   5  15   5]
 [-12   6   9  13   2  -8   3   5]
 [  5  19   5   4   9   1  11   1]]


[[ -1  11  16  23   5   5  15   5]
 [-12   6   9  13   2  -8   3   5]
 [  5  19   5   4   9   1  11   1]]




In [12]:
import numpy as np
import pandas as pd

conv_times = [1,2,3]
conv_times2 =[4,5,6]

conv_repeats = [conv_times, conv_times2]
conv_repeats2 = [conv_times2, conv_times]

one_thread = np.array([conv_repeats, conv_repeats2])

In [ ]:
import multiprocessing.process
from PIL import Image
import numpy as np
import multiprocessing
import time
import cv2

kernel_relief = np.array([[-2, -1, 0],
                          [-1,  1, 1],
                          [ 0,  1, 2]])

images = ['collage 10240.png','cruella 20480.jpg','iceland 12800.jpg']

padding = 0
stride = 1

threads_quantities = [4, 6]
result_times = [] # здесь будут временные затраты на все операции


# Разделение матрицы на две части
def split_matrix(matrix, parts_quantity):
    indexes = []
    parts = []
    for i in range(parts_quantity+1):
        indexes.append((matrix.shape[0]//parts_quantity)*i)

    i = 0
    while (i < parts_quantity):
        parts.append(matrix[indexes[i]:indexes[i+1]])
        i += 1
    
    return parts

def get_rgb_matrices(image_path):
    image = Image.open(image_path)
    image_np = np.array(image)
    red_channel = green_channel = blue_channel = 0
    if (image_np.ndim == 3):
        red_channel = image_np[:,:,0]
        green_channel = image_np[:,:,1]
        blue_channel = image_np[:,:,2]
    return (red_channel, green_channel, blue_channel)


def constant_zero_padding(input_arr):
    """
    function to generate 0 padding around input array

    Parameter
    ---------
    input_arr : arr
        The array to which padding wil be added
    
    Returns
    -------
    output : arr
        The input_arr with paddings
    """
    output = np.zeros((input_arr.shape[0]+2, input_arr.shape[1]+2), dtype=int)

    for i in range(input_arr.shape[0]):
        for j in range(input_arr.shape[1]):
            output[i+1][j+1] = input_arr[i,j]
    return output

def convolution(kernel, input_arr, padding, stride):
    """
    function which produces convuliton operation on input_arr
    
    Parameters
    ----------
    kernel : arr
        The kernel which used in convolution, must be smaller than input_arr
    input_arr : arr
        The array which will be convoluted, must be bigger than kernel
    padding : int
        '1' is to create const zero padding around array in order to make 
        output array the same scale as the input one and '0' to not do it.
    stride : int
        step by which the convolution opeartion is shifted through 
        input_arr. But for now it's uses only to define output array sizes

    Returns
    -------
    output_arr
        convoluted array
    """
    frame = np.zeros_like(kernel)
    kernel_size = kernel.shape[0]
    rows_num = input_arr.shape[0]
    column_num = input_arr.shape[1]
    output_height=((rows_num-kernel_size+2*padding)/stride)+1
    output_width=((column_num-kernel_size+2*padding)/stride)+1
    output_arr = np.zeros((int(output_height),int(output_width)), dtype=int)
    conv_times = []

    i = j = k = l = val = 0

    while k < output_arr.shape[0]:
        while l < output_arr.shape[1]:
            while i < kernel_size:
                while j < kernel_size:
                    frame[i][j] = input_arr[i+k][j+l] 
                    j += 1

                j = 0
                i += 1

            if (i == kernel_size): 
                for m in range(frame.shape[0]):
                    for n in range(frame.shape[1]):
                        val += frame[m][n]*kernel_relief[m][n]  
            output_arr[k][l] = val
            val = j = i = 0
            l += 1         
        l = 0
        k += 1
    return output_arr

def not_mine_convolution(index, image_part, kernel, result):
    print(f"Start processing part {index}")
    convoluted_image = cv2.filter2D(image_part, -1, kernel)
    result[index] = convoluted_image


for threads_quantity in threads_quantities:
    if __name__ == '__main__':
        conv_times_arr = [] # здесь будут храниться convs_times 5 раз для точности
        for i in range(5):
            convs_times = [] # здесь будут время свёртки каждого изображения в итерацию, то есть размер массива равен 3
            for image_name in images:
                image = cv2.imread(image_name, cv2.IMREAD_COLOR)
                print(f'image: {image_name}, image shape: {image.shape}')
                parts = split_matrix(image, threads_quantity)
                print(f'parts count: {len(parts)}, shape of one part: {parts[0].shape}')
                
                start_time = time.time()
                with multiprocessing.Manager() as manager:
                    processes = []
                    # Инициализация списка для результатов с фиксированным размером
                    result_list = manager.list([None] * threads_quantity)

                    print('final')
                    
                    for index, part in enumerate(parts):
                        process = multiprocessing.Process(target=not_mine_convolution, args=(index, part, kernel_relief, result_list))
                        processes.append(process)

                    print('final2')
                    for process in processes:
                        process.start()

                    print('final3')
                    for process in processes:
                        process.join()
                    
                    final_image = np.vstack(list(result_list))
                    end_time = time.time()
                    conv_times.append(end_time - start_time)
                
                
                final_image_name = image_name + 'convoluted.jpg'
                cv2.imwrite(final_image_name, final_image)
            conv_times_arr.append(convs_times)



        print(f'processes: {threads_quantity}')
        for conv_time in conv_times_arr:
            print(f't = {conv_time}')

    result_times.append(conv_times_arr)
    

image: collage 10240.png, image shape: (7680, 10240, 3)
parts count: 4, shape of one part: (1920, 10240, 3)
final
final2


In [ ]:
threads_quantity = 2

result_times = []



x_label = (str(threads_quantity) + '_thread(s)')
print(x_label)

2_thread(s)


In [18]:
pip install opencv-python

Defaulting to user installation because normal site-packages is not writeable
   ---------------------------------------- 0.0/38.8 MB ? eta -:--:--
   ---------------------------------------- 0.0/38.8 MB ? eta -:--:--
    --------------------------------------- 0.5/38.8 MB 2.8 MB/s eta 0:00:14
    --------------------------------------- 0.8/38.8 MB 2.8 MB/s eta 0:00:14
   - -------------------------------------- 1.6/38.8 MB 2.5 MB/s eta 0:00:16
   -- ------------------------------------- 2.1/38.8 MB 2.6 MB/s eta 0:00:15
   -- ------------------------------------- 2.9/38.8 MB 2.6 MB/s eta 0:00:14
   --- ------------------------------------ 3.4/38.8 MB 2.7 MB/s eta 0:00:14
   ---- ----------------------------------- 3.9/38.8 MB 2.6 MB/s eta 0:00:14
   ---- ----------------------------------- 4.5/38.8 MB 2.6 MB/s eta 0:00:14
   ----- ---------------------------------- 5.2/38.8 MB 2.7 MB/s eta 0:00:13
   ----- ---------------------------------- 5.8/38.8 MB 2.7 MB/s eta 0:00:13
   ------ --